# Interactive monitoring of cells characteristics

Here we want to use the contours produced from the images segmentation, and the tracking results, to be able to investigate the cells' characteristics in a user-friendly fashion, via the Napari GUI.

In [1]:
import cv2
import imageio
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import scipy
from skimage import measure # to get contours from masks

# btrack module and configuration file
import btrack
from btrack.dataio import localizations_to_objects
from btrack.constants import BayesianUpdates
from btrack.render import plot_tracks

import imagecodecs
import napari

C:\ProgramData\Anaconda3\lib\site-packages\napari\_qt\__init__.py:37: UserWarning: 
    napari was tested with QT library `>=5.12.3`.
    The version installed is 5.9.7. Please report any issues with this
    specific QT version at https://github.com/Napari/napari/issues.
    
  warn(message=warn_message)


# Load data

Here we load several things:
* Bright Field images
* Predictions made by the model
* Corresponding RFP images

As well as:
* The dataframe containing the tracking results
* The array (sorted; with the same rows order as in the dataframe) containing the cells contours at each time step.
Those two files were generated in the ```Perform_Tracking.ipynb``` notebook.

In [2]:
os.chdir("D:/Hugo/Whole_Cell")  # root dir containg Predictions and Images Path

predictions = imageio.volread("Predictions/Predictions_UNET110_ep50_f0001_1-20.tif") # neural network predictions (segmentation)
corresponding_imgs = imageio.volread("Test_Set/BF_f0001-1_20imgs.tif") # bf images
rfp_imgs = imageio.volread("D:/Hugo/Data/F0001/RFP_f0001-1-20.tif") # rfp images

tracks_and_contours = pd.read_csv("D:/Hugo/Python_Scripts/Notebooks/Tracking_Results.csv") # tracking positions and the corresponding contours
contours = np.load("D:/Hugo/Python_Scripts/Notebooks/contours.npy", allow_pickle=True)

# a bit dirty... I have to replace the contours in dataframe because if loaded from a csv, it will be a string and not an array
# so we save it as a separate array (but with god rows order, see Perform_Tracking.py) and load and cast it to float-32 (for opencv)
contours32 = []
for c in contours:
    cont = []
    for pos in c:
        cont.append(pos.astype(np.float32))
    contours32.append(np.array(cont))
contours32 = np.array(contours32)

# put the "good" contours i.e. the array, and not the loaded string
tracks_and_contours = tracks_and_contours.drop("Contours", axis=1)
tracks_and_contours["Contours"] = contours32

print(tracks_and_contours.shape, contours.shape)
tracks_and_contours.head()

FileNotFoundError: No such file: 'D:\Hugo\Whole_Cell\Predictions\Predictions_UNET110_ep50_f0001_1-20.tif'

### First examples

In [ ]:
# retrieve the contours of a specified cell, for all frame
ID = 38
contours = tracks_and_contours[tracks_and_contours["ID"] == ID][["Frame", "Contours"]]
print(contours)

In [ ]:
# retrieve the contours of a specified cell, at a specified frame
ID, frame = 38, 19
contours = tracks_and_contours[(tracks_and_contours["ID"] == ID) & (tracks_and_contours["Frame"] == frame)]["Contours"]
print(f"Contours of cell {ID} at frame {frame} : {contours}")

## Interactive viewer in Napari

Here we implement functions to view the predictions and tracking results using the Napari interface. This interface even allows to go further, by allowing the user to click on the cell on which she/he wants to investigate, by plotting several characteristics (surface, etc.).

In [ ]:
def plot_cells_characteristics(tracks_contours, ID, characteristics=["Surface"]):
    """
    Given the ID of a cell, plots the demanded characteristics over time.
    tracks_contours (pd.DataFrame): df containing the tracks and the contours of all cells at all frames (ID, Frame, X, Y, Contours)
    ID (int): ID of the cell(s) to track. TODO : extend to take a list of IDs instead of a single one.
    characteristics (list<str>): List of the characteristics to plot
    start_frame (int): first frame to display (included)
    end_frame (int): last frame to display (excluded)
    """
    cell = tracks_contours[(tracks_contours["ID"] == ID)]

    fig, ax = plt.subplots(1, len(characteristics), figsize=(20, 10))
    if "Surface" in characteristics:
        surfaces_per_frame = retrieve_surfaces(cell) # dataframe containing the cell's surface over time (frame by frame)
        start_frame, end_frame = 1000, -1
        for ID in pd.unique(surfaces_per_frame["ID"]): # for each cell, plot its surface over time
            start_frame, end_frame = np.min([start_frame, np.min(surfaces_per_frame["Frame"])]), np.max([end_frame, np.max(surfaces_per_frame["Frame"])])
            if len(characteristics) > 1:
                ax[0].plot(surfaces_per_frame["Frame"], surfaces_per_frame["Surface"], label=ID)
                ax[0].set_xlim(left=start_frame, right=end_frame)
                ax[0].set_title(f"Surface of cell {ID} between frame {start_frame} and {end_frame}")
                plt.xticks(range(int(start_frame), int(end_frame) + 1))
            else:
                ax.plot(surfaces_per_frame["Frame"], surfaces_per_frame["Surface"], label=ID)
                ax.set_xlim(left=np.min(surfaces_per_frame["Frame"]), right=np.max(surfaces_per_frame["Frame"]))
                ax.set_title(f"Surface of cell {ID} between frame {start_frame} and {end_frame}")
                plt.xticks(range(int(start_frame), int(end_frame) + 1))

    plt.show()
    return

def retrieve_surfaces(cell):
    """
    Retrieve the surface of a cell, whose contours are given in the dataframe cell.
    cell (pd.DataFrame): df containing the contours of the query cell.
    """
    surfaces_over_time = []
    for i, line in cell.iterrows():
        s = cv2.contourArea(line["Contours"])
        surfaces_over_time.append({"ID": line["ID"], "Frame": line["Frame"], "Surface": s})
    return pd.DataFrame(surfaces_over_time)

In [ ]:
resized = []
for im in corresponding_imgs:
    resized.append(cv2.resize(im, (512, 512)))
resized = np.array(resized)

with napari.gui_qt():
    viewer = napari.view_image(resized, name="BF Images")
    rfp_layer = viewer.add_image(rfp_imgs, blending="additive", colormap="red", name="RFP Images")
    images_layer = viewer.add_image(predictions, opacity=0.5, colormap="bop blue", blending="additive", name="Predictions")
    viewer.add_tracks(tracks_and_contours[["ID", "Frame", "X", "Y"]], name="Tracks")
    IDs = []
    
    @viewer.mouse_drag_callbacks.append
    def cells_characteristics(img, event):
        """
        Encapsulate the call to plot_cells_characteristics and implements the user graphical interactions with the Napari GUI. 
        """
        frame, x, y = np.round(images_layer.coordinates).astype(int)
        
        def get_closest_cell(frame, x, y):
            """
            Find the closest cell to click point
            Return : ID of the track corresponding to the closest cell
            """
            distances = []
            for _, c in tracks_and_contours[tracks_and_contours["Frame"] == frame].iterrows():
                distances.append(np.sqrt((c["X"] - x) ** 2 + (c["Y"] - y) ** 2))
            distances = np.array(distances)
            min_dist, min_dist_arg = np.min(distances), np.argmin(distances)
            if min_dist < 12:
                closest_cell = tracks_and_contours[tracks_and_contours["Frame"] == frame].iloc[min_dist_arg] # retrieve the closes cell
            else: # don't get the closest, the click was just to move on the map
                return -1
            return closest_cell["ID"]
        
        ID = get_closest_cell(frame, x, y)
        
        if ID == -1:
            return
        else:
            IDs.append(ID)
            print(IDs)
#             viewer.add_shapes(tracks_and_contours[(tracks_and_contours["ID"] == ID)]["Contours"].to_numpy(), shape_type="polygon")
            plot_cells_characteristics(tracks_and_contours, ID)